Connect to your miro board.

See https://developers.miro.com/docs/getting-started for miro setup instructions

In [ ]:
import requests
import json

url = """YOUR BOARD URL""" # e.g. https://api.miro.com/v1/boards/XXXXXXXXXXX"
bearer_token = """YOUR TOKEN"""
headers = {"Authorization": bearer_token}

In [ ]:
# Access board content with the widgets endpoint
# Get the content as a json object

widgets_url = url + "/widgets/"
response = requests.request("GET", widgets_url, headers=headers)
all_widgets = response.json()

In [ ]:
# Have a look at an entry

all_widgets['data'][0]

Separate the data into concepts (the text boxes with your mindmap content) and lines (the lines joining your textboxes).

 - 'concepts' is a dictionary whose keys are the miro widget IDs of the text boxes. The values are dictionaries with single key 'text' containing the text content of the boxes.

 - 'lines' is a dictionary whose keys are the miro widget IDs of the lines. The values are dictionaries with 'start_id' and 'end_id' keys containing the miro widget IDs of the boxes that the line joins.
 
 - 'endPoints' is a list of 'end_id's, used below to identify the mindmap starting node.

In [ ]:
concepts = {}
lines = {}
endPoints = []

for i in all_widgets['data']:
    if i['type'] == "text":
        concepts[i['id']] = {'text':i['text']}
    if i['type'] == "line":
        lines[i['id']] = {'start_id':i['startWidget']['id'], 'end_id':i['endWidget']['id']}
        endPoints.append(i['endWidget']['id'])

Create the mind_map object and identify the starting node (the central concept from which the other concepts branch).

The starting node is the concept which is not an end point for any lines.

In [ ]:
mind_map = {}

for i in concepts.keys():
    if i in endPoints:
        pass
    else:
        mind_map[i] = {'text':concepts[i]['text'], 'branches':{}}

The get_branches function expands a given node. That is:
 - it checks if there are any lines that have the current node as a start point
 - if there are, it gets the connected concept and adds it as a 'branch' of the given node
 - it returns the branch nodes for further iteration

In [ ]:
def get_branches(node_id, mind_map_node, lines, concepts):
    new_nodes = []
    for i in lines:
        if lines[i]['start_id'] == node_id:
            branch_id = lines[i]['end_id']
            mind_map_node['branches'][branch_id] = {'text':concepts[lines[i]['end_id']]['text'], 'branches':{}}
            new_nodes.append([branch_id, mind_map_node['branches'][branch_id]])
    return new_nodes

Recursively build the mind_map object:
 - initiate a stack with the starting node
 - pop the node from the stack and expand its branches
 - add any branches back to the stack
 - repeat until the stack is empty

In [ ]:
stack = []

# Confirm there is a single start point

if len(list(mind_map.keys())) != 1:
    print("error single start point required")
else:
    start = []
    start.extend(mind_map.keys())
    start.append(mind_map[start[0]])
    stack.append(start)
    
while stack:
    current = stack.pop(0)
    new_nodes = get_branches(current[0], current[1], lines, concepts)
    stack.extend(new_nodes)

Export the mind_map as a json object.

In [ ]:
with open("mind_map.json", "w") as outfile:  
    json.dump(mind_map, outfile) 